# Air Quality Data Exploration - Updated for Your Data Structure

This notebook provides exploratory data analysis for your specific air quality forecasting dataset with columns:
- year, month, day, hour
- O3_forecast, NO2_forecast (targets)
- T_forecast, q_forecast, u_forecast, v_forecast, w_forecast (meteorological)
- NO2_satellite, HCHO_satellite, ratio_satellite (satellite data)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Add project root to path
import sys
sys.path.append('..')

from config.settings import settings
from src.utils.logger import setup_logging, get_logger

# Setup logging
setup_logging()
logger = get_logger(__name__)

## 1. Load and Inspect Data

In [ ]:
# Load your dataset
data_path = '../data/raw/Raw Data/Data_SIH_2025/site_1_unseen_input_data.csv'

# Check if file exists
if Path(data_path).exists():
    df = pd.read_csv(data_path)
    print(f"Data loaded successfully: {df.shape}")
    
    # Create datetime from components
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])
    
else:
    print(f"Data file not found: {data_path}")
    print("Creating sample data with your structure for demonstration...")
    
    # Create sample data matching your structure
    dates = pd.date_range('2020-01-01', '2023-12-31', freq='H')
    n_samples = len(dates)
    
    # Generate synthetic data with your column structure
    np.random.seed(42)
    
    df = pd.DataFrame({
        'year': dates.year,
        'month': dates.month,
        'day': dates.day,
        'hour': dates.hour,
        'O3_forecast': np.random.lognormal(3.8, 0.4, n_samples),
        'NO2_forecast': np.random.lognormal(3.5, 0.5, n_samples),
        'T_forecast': np.random.normal(25, 8, n_samples),  # Celsius
        'q_forecast': np.random.beta(2, 5, n_samples) * 0.02,  # Specific humidity
        'u_forecast': np.random.normal(0, 3, n_samples),  # U wind component
        'v_forecast': np.random.normal(0, 3, n_samples),  # V wind component
        'w_forecast': np.random.normal(0, 0.1, n_samples),  # W wind component
        'NO2_satellite': np.random.lognormal(3.3, 0.6, n_samples),
        'HCHO_satellite': np.random.lognormal(2.8, 0.4, n_samples),
        'ratio_satellite': np.random.lognormal(0.5, 0.3, n_samples)
    })
    
    # Create datetime column
    df['datetime'] = dates
    
    print(f"Sample data created: {df.shape}")

In [ ]:
# Basic information about the dataset
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"Columns: {list(df.columns)}")

# Display first few rows
df.head()

In [ ]:
# Data types and missing values
print("Data Types and Missing Values:")
info_df = pd.DataFrame({
    'Data Type': df.dtypes,
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df)) * 100
})
print(info_df)

## 2. Statistical Summary

In [ ]:
# Statistical summary for numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols].describe()

## 3. Target Variables Analysis (O3_forecast, NO2_forecast)

In [ ]:
# Distribution of target variables
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# NO2_forecast distribution
axes[0, 0].hist(df['NO2_forecast'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('NO2_forecast Distribution')
axes[0, 0].set_xlabel('NO2_forecast (μg/m³)')
axes[0, 0].set_ylabel('Frequency')

# O3_forecast distribution
axes[0, 1].hist(df['O3_forecast'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('O3_forecast Distribution')
axes[0, 1].set_xlabel('O3_forecast (μg/m³)')
axes[0, 1].set_ylabel('Frequency')

# NO2_forecast box plot
axes[1, 0].boxplot(df['NO2_forecast'])
axes[1, 0].set_title('NO2_forecast Box Plot')
axes[1, 0].set_ylabel('NO2_forecast (μg/m³)')

# O3_forecast box plot
axes[1, 1].boxplot(df['O3_forecast'])
axes[1, 1].set_title('O3_forecast Box Plot')
axes[1, 1].set_ylabel('O3_forecast (μg/m³)')

plt.tight_layout()
plt.show()

In [ ]:
# Correlation between NO2_forecast and O3_forecast
plt.figure(figsize=(8, 6))
plt.scatter(df['NO2_forecast'], df['O3_forecast'], alpha=0.5)
plt.xlabel('NO2_forecast (μg/m³)')
plt.ylabel('O3_forecast (μg/m³)')
plt.title('NO2_forecast vs O3_forecast Correlation')

# Add correlation coefficient
correlation = df['NO2_forecast'].corr(df['O3_forecast'])
plt.text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
         transform=plt.gca().transAxes, fontsize=12,
         bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.grid(True, alpha=0.3)
plt.show()

## 4. Temporal Patterns

In [ ]:
# Add day of week for analysis
df['day_of_week'] = df['datetime'].dt.dayofweek

In [ ]:
# Time series plot
fig, axes = plt.subplots(2, 1, figsize=(15, 10))

# Plot subset of data for clarity
subset = df.iloc[:24*30]  # First 30 days

axes[0].plot(subset['datetime'], subset['NO2_forecast'], label='NO2_forecast', alpha=0.8)
axes[0].set_title('NO2_forecast Time Series (First 30 Days)')
axes[0].set_ylabel('NO2_forecast (μg/m³)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(subset['datetime'], subset['O3_forecast'], label='O3_forecast', color='orange', alpha=0.8)
axes[1].set_title('O3_forecast Time Series (First 30 Days)')
axes[1].set_ylabel('O3_forecast (μg/m³)')
axes[1].set_xlabel('Date')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Hourly patterns
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# NO2_forecast hourly pattern
hourly_no2 = df.groupby('hour')['NO2_forecast'].mean()
axes[0].plot(hourly_no2.index, hourly_no2.values, marker='o')
axes[0].set_title('Average NO2_forecast by Hour of Day')
axes[0].set_xlabel('Hour')
axes[0].set_ylabel('NO2_forecast (μg/m³)')
axes[0].grid(True, alpha=0.3)

# O3_forecast hourly pattern
hourly_o3 = df.groupby('hour')['O3_forecast'].mean()
axes[1].plot(hourly_o3.index, hourly_o3.values, marker='o', color='orange')
axes[1].set_title('Average O3_forecast by Hour of Day')
axes[1].set_xlabel('Hour')
axes[1].set_ylabel('O3_forecast (μg/m³)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Monthly patterns
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# NO2_forecast monthly pattern
monthly_no2 = df.groupby('month')['NO2_forecast'].mean()
axes[0].plot(monthly_no2.index, monthly_no2.values, marker='o')
axes[0].set_title('Average NO2_forecast by Month')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('NO2_forecast (μg/m³)')
axes[0].set_xticks(range(1, 13))
axes[0].grid(True, alpha=0.3)

# O3_forecast monthly pattern
monthly_o3 = df.groupby('month')['O3_forecast'].mean()
axes[1].plot(monthly_o3.index, monthly_o3.values, marker='o', color='orange')
axes[1].set_title('Average O3_forecast by Month')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('O3_forecast (μg/m³)')
axes[1].set_xticks(range(1, 13))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Meteorological Variables Analysis

In [ ]:
# Correlation matrix
met_vars = ['T_forecast', 'q_forecast', 'u_forecast', 'v_forecast', 'w_forecast']
satellite_vars = ['NO2_satellite', 'HCHO_satellite', 'ratio_satellite']
target_vars = ['NO2_forecast', 'O3_forecast']

corr_matrix = df[met_vars + satellite_vars + target_vars].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Correlation Matrix: All Variables')
plt.tight_layout()
plt.show()

In [ ]:
# Wind analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Wind speed calculation
df['wind_speed'] = np.sqrt(df['u_forecast']**2 + df['v_forecast']**2)
df['wind_direction'] = np.arctan2(df['v_forecast'], df['u_forecast']) * 180 / np.pi
df['wind_direction'] = (df['wind_direction'] + 360) % 360

# Wind speed distribution
axes[0, 0].hist(df['wind_speed'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('Wind Speed Distribution')
axes[0, 0].set_xlabel('Wind Speed (m/s)')
axes[0, 0].set_ylabel('Frequency')

# Wind direction distribution
axes[0, 1].hist(df['wind_direction'], bins=36, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Wind Direction Distribution')
axes[0, 1].set_xlabel('Wind Direction (degrees)')
axes[0, 1].set_ylabel('Frequency')

# Vertical wind component
axes[1, 0].hist(df['w_forecast'], bins=50, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Vertical Wind Component Distribution')
axes[1, 0].set_xlabel('w_forecast (m/s)')
axes[1, 0].set_ylabel('Frequency')

# Temperature distribution
axes[1, 1].hist(df['T_forecast'], bins=50, alpha=0.7, edgecolor='black')
axes[1, 1].set_title('Temperature Distribution')
axes[1, 1].set_xlabel('T_forecast (°C or K)')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 6. Satellite Data Analysis

In [ ]:
# Satellite variables distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# NO2_satellite distribution
axes[0, 0].hist(df['NO2_satellite'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 0].set_title('NO2_satellite Distribution')
axes[0, 0].set_xlabel('NO2_satellite')
axes[0, 0].set_ylabel('Frequency')

# HCHO_satellite distribution
axes[0, 1].hist(df['HCHO_satellite'], bins=50, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('HCHO_satellite Distribution')
axes[0, 1].set_xlabel('HCHO_satellite')
axes[0, 1].set_ylabel('Frequency')

# ratio_satellite distribution
axes[1, 0].hist(df['ratio_satellite'], bins=50, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('ratio_satellite Distribution')
axes[1, 0].set_xlabel('ratio_satellite')
axes[1, 0].set_ylabel('Frequency')

# Satellite vs Forecast comparison
axes[1, 1].scatter(df['NO2_satellite'], df['NO2_forecast'], alpha=0.5)
axes[1, 1].set_title('NO2_satellite vs NO2_forecast')
axes[1, 1].set_xlabel('NO2_satellite')
axes[1, 1].set_ylabel('NO2_forecast')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Feature Relationships

In [ ]:
# Scatter plots: Key variables vs targets
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

key_vars = ['T_forecast', 'wind_speed', 'NO2_satellite']

for i, var in enumerate(key_vars):
    # vs NO2_forecast
    axes[0, i].scatter(df[var], df['NO2_forecast'], alpha=0.3, s=1)
    axes[0, i].set_xlabel(var)
    axes[0, i].set_ylabel('NO2_forecast')
    axes[0, i].set_title(f'{var} vs NO2_forecast')
    axes[0, i].grid(True, alpha=0.3)
    
    # vs O3_forecast
    axes[1, i].scatter(df[var], df['O3_forecast'], alpha=0.3, s=1, color='orange')
    axes[1, i].set_xlabel(var)
    axes[1, i].set_ylabel('O3_forecast')
    axes[1, i].set_title(f'{var} vs O3_forecast')
    axes[1, i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Data Quality Assessment

In [ ]:
# Check for outliers using IQR method
def detect_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return len(outliers), (len(outliers) / len(df)) * 100

print("Outlier Analysis:")
all_vars = target_vars + met_vars + satellite_vars
for col in all_vars:
    outlier_count, outlier_pct = detect_outliers(df, col)
    print(f"{col}: {outlier_count} outliers ({outlier_pct:.2f}%)")

In [ ]:
# Check for data gaps
df_sorted = df.sort_values('datetime')
time_diff = df_sorted['datetime'].diff()
expected_freq = pd.Timedelta(hours=1)  # Assuming hourly data

gaps = time_diff[time_diff > expected_freq]
print(f"Data gaps found: {len(gaps)}")

if len(gaps) > 0:
    print("\nLargest gaps:")
    print(gaps.nlargest(5))

## 9. Summary and Recommendations

In [ ]:
print("DATA EXPLORATION SUMMARY")
print("=" * 50)
print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['datetime'].min()} to {df['datetime'].max()}")
print(f"Total missing values: {df.isnull().sum().sum()}")
print(f"Missing percentage: {(df.isnull().sum().sum() / df.size) * 100:.2f}%")

print("\nTARGET VARIABLES:")
print(f"NO2_forecast - Mean: {df['NO2_forecast'].mean():.2f}, Std: {df['NO2_forecast'].std():.2f}, Range: [{df['NO2_forecast'].min():.2f}, {df['NO2_forecast'].max():.2f}]")
print(f"O3_forecast - Mean: {df['O3_forecast'].mean():.2f}, Std: {df['O3_forecast'].std():.2f}, Range: [{df['O3_forecast'].min():.2f}, {df['O3_forecast'].max():.2f}]")
print(f"NO2-O3 Correlation: {df['NO2_forecast'].corr(df['O3_forecast']):.3f}")

print("\nKEY FEATURES:")
print(f"Temperature range: {df['T_forecast'].min():.1f} to {df['T_forecast'].max():.1f}")
print(f"Wind speed range: 0 to {df['wind_speed'].max():.1f} m/s")
print(f"Satellite NO2 vs Forecast correlation: {df['NO2_satellite'].corr(df['NO2_forecast']):.3f}")

print("\nRECOMMENDATIONS:")
print("1. Handle missing values using appropriate imputation methods")
print("2. Consider outlier treatment for extreme values")
print("3. Engineer additional features from wind components (speed, direction)")
print("4. Create lag features for time series modeling")
print("5. Normalize/standardize features before modeling")
print("6. Leverage satellite data correlation with forecasts")
print("7. Consider seasonal and diurnal patterns in feature engineering")